In [44]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import concat, lit

# Spark Configuration
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Football_Pipeline")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# Create the Spark session
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup Hadoop configuration for GCS
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")


playersDF = spark.read.format("csv").option("header", "true") \
    .load("gs://data_assignment2/players_table.csv")
teamsDF = spark.read.format("csv").option("header", "true") \
    .load("gs://data_assignment2/teams_table.csv")
performanceDF = spark.read.format("csv").option("header", "true") \
    .load("gs://data_assignment2/player_performance_table.csv")


playersDF.show(5)
teamsDF.show(5)
performanceDF.show(5)


+-----------------+-------+----+-----+--------------+
|           Player| Nation| Age|  Pos|         Squad|
+-----------------+-------+----+-----+--------------+
|       Max Aarons|eng ENG|23.0|   DF|   Bournemouth|
| Brenden Aaronson| us USA|22.0|MF,FW|  Union Berlin|
|  Paxten Aaronson| us USA|19.0|   MF|Eint Frankfurt|
|Keyliane Abdallah| fr FRA|17.0|   FW|     Marseille|
| Yunis Abdelhamid| ma MAR|35.0|   DF|         Reims|
+-----------------+-------+----+-----+--------------+
only showing top 5 rows

+--------------+------------------+
|         Squad|              Comp|
+--------------+------------------+
|   Bournemouth|eng Premier League|
|  Union Berlin|     de Bundesliga|
|Eint Frankfurt|     de Bundesliga|
|     Marseille|        fr Ligue 1|
|         Reims|        fr Ligue 1|
+--------------+------------------+
only showing top 5 rows

+-----------------+--------------+---+------+----+---+---+---+---+---+
|           Player|         Squad| MP|Starts| Min|Gls|Ast|G+A| xG|xAG

In [45]:
from pyspark.sql.functions import col

# Relevant columns from performanceDF
filteredPerformanceDF = performanceDF.select(
    "Player", "Squad", "MP", "Gls", "Ast", "G+A", "xG", "xAG"
).na.drop("any", subset=["Player", "Squad", "MP", "G+A"])


filteredPerformanceDF.printSchema()
filteredPerformanceDF.show(5)


root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- Gls: string (nullable = true)
 |-- Ast: string (nullable = true)
 |-- G+A: string (nullable = true)
 |-- xG: string (nullable = true)
 |-- xAG: string (nullable = true)

+-----------------+--------------+---+---+---+---+---+---+
|           Player|         Squad| MP|Gls|Ast|G+A| xG|xAG|
+-----------------+--------------+---+---+---+---+---+---+
|       Max Aarons|   Bournemouth| 20|  0|  1|  1|0.0|0.8|
| Brenden Aaronson|  Union Berlin| 30|  2|  2|  4|2.0|1.9|
|  Paxten Aaronson|Eint Frankfurt|  7|  0|  1|  1|0.1|0.1|
|Keyliane Abdallah|     Marseille|  1|  0|  0|  0|0.0|0.0|
| Yunis Abdelhamid|         Reims| 31|  4|  0|  4|3.4|0.3|
+-----------------+--------------+---+---+---+---+---+---+
only showing top 5 rows



In [46]:
from pyspark.sql.functions import concat, lit

# Keep relevant columns from playersDF
filteredPlayersDF = playersDF.select(
    "Player", "Nation", "Age", "Pos", "Squad"
).na.drop("any", subset=["Player", "Squad","Age"])


filteredPlayersDF = filteredPlayersDF.withColumn(
    "Player_Name", concat(col("Player"), lit(" "))
)


filteredPlayersDF.printSchema()
filteredPlayersDF.show(5)


root
 |-- Player: string (nullable = true)
 |-- Nation: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- Player_Name: string (nullable = true)

+-----------------+-------+----+-----+--------------+------------------+
|           Player| Nation| Age|  Pos|         Squad|       Player_Name|
+-----------------+-------+----+-----+--------------+------------------+
|       Max Aarons|eng ENG|23.0|   DF|   Bournemouth|       Max Aarons |
| Brenden Aaronson| us USA|22.0|MF,FW|  Union Berlin| Brenden Aaronson |
|  Paxten Aaronson| us USA|19.0|   MF|Eint Frankfurt|  Paxten Aaronson |
|Keyliane Abdallah| fr FRA|17.0|   FW|     Marseille|Keyliane Abdallah |
| Yunis Abdelhamid| ma MAR|35.0|   DF|         Reims| Yunis Abdelhamid |
+-----------------+-------+----+-----+--------------+------------------+
only showing top 5 rows



In [47]:
# Relevant columns from teamsDF
filteredTeamsDF = teamsDF.select(
    "Squad", "Comp"
).na.drop("any", subset=["Squad", "Comp"])


filteredTeamsDF.printSchema()
filteredTeamsDF.show(5)


root
 |-- Squad: string (nullable = true)
 |-- Comp: string (nullable = true)

+--------------+------------------+
|         Squad|              Comp|
+--------------+------------------+
|   Bournemouth|eng Premier League|
|  Union Berlin|     de Bundesliga|
|Eint Frankfurt|     de Bundesliga|
|     Marseille|        fr Ligue 1|
|         Reims|        fr Ligue 1|
+--------------+------------------+
only showing top 5 rows



In [48]:
# Join Performance and Players tables
performance_playersDF = filteredPerformanceDF.join(
    filteredPlayersDF,
    on=["Player", "Squad"],
    how="inner"
)

# Join with Teams table
finalTableDF = performance_playersDF.join(
    filteredTeamsDF,
    on=["Squad"],
    how="inner"
)


finalTableDF = finalTableDF.select("Player", "Squad", "MP", "G+A", "Age", "Comp")


finalTableDF.printSchema()
finalTableDF.show(10)


root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- G+A: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Comp: string (nullable = true)

+-----------------+--------------+---+---+----+------------------+
|           Player|         Squad| MP|G+A| Age|              Comp|
+-----------------+--------------+---+---+----+------------------+
|       Max Aarons|   Bournemouth| 20|  1|23.0|eng Premier League|
| Brenden Aaronson|  Union Berlin| 30|  4|22.0|     de Bundesliga|
|  Paxten Aaronson|Eint Frankfurt|  7|  1|19.0|     de Bundesliga|
|Keyliane Abdallah|     Marseille|  1|  0|17.0|        fr Ligue 1|
| Yunis Abdelhamid|         Reims| 31|  4|35.0|        fr Ligue 1|
|Salis Abdul Samed|          Lens| 27|  0|23.0|        fr Ligue 1|
|    Nabil Aberdin|        Getafe|  2|  0|20.0|        es La Liga|
|  Laurent Abergel|       Lorient| 33|  3|30.0|        fr Ligue 1|
|   Matthis Abline|        Nantes| 22|  5|

In [49]:
from pyspark.sql.functions import col

# Filter out players with fewer than 5 matches
filteredTableDF = finalTableDF.filter(col("MP") >= 5)

# Schema after filtering
filteredTableDF.printSchema()
filteredTableDF.show(10)

root
 |-- Player: string (nullable = true)
 |-- Squad: string (nullable = true)
 |-- MP: string (nullable = true)
 |-- G+A: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Comp: string (nullable = true)

+-----------------+--------------+---+---+----+------------------+
|           Player|         Squad| MP|G+A| Age|              Comp|
+-----------------+--------------+---+---+----+------------------+
|       Max Aarons|   Bournemouth| 20|  1|23.0|eng Premier League|
| Brenden Aaronson|  Union Berlin| 30|  4|22.0|     de Bundesliga|
|  Paxten Aaronson|Eint Frankfurt|  7|  1|19.0|     de Bundesliga|
| Yunis Abdelhamid|         Reims| 31|  4|35.0|        fr Ligue 1|
|Salis Abdul Samed|          Lens| 27|  0|23.0|        fr Ligue 1|
|  Laurent Abergel|       Lorient| 33|  3|30.0|        fr Ligue 1|
|   Matthis Abline|        Nantes| 22|  5|20.0|        fr Ligue 1|
|            Abner|         Betis| 23|  1|23.0|        es La Liga|
|Zakaria Aboukhlal|      Toulouse| 13|  3|

In [50]:
from pyspark.sql.functions import col, sum, expr, round

# Aggregate by Player and League with rounding to 2 decimals
groupedResultsDF = filteredTableDF.groupBy("Player", "Comp","Squad").agg(
    round(sum("G+A"), 2).alias("total_G+A"),  # Round total G+A to 2 decimals
    round(sum("G+A") / sum("MP"), 2).alias("G+A_per_match")  # Round G+A per match to 2 decimals
)

# Order by League and G+A per Match
orderedResultsDF = groupedResultsDF.orderBy(col("Comp").asc(), col("G+A_per_match").desc())

#
orderedResultsDF.show(20)


+-----------------+-------------+--------------+---------+-------------+
|           Player|         Comp|         Squad|total_G+A|G+A_per_match|
+-----------------+-------------+--------------+---------+-------------+
|       Harry Kane|de Bundesliga| Bayern Munich|     44.0|         1.38|
|  Serhou Guirassy|de Bundesliga|     Stuttgart|     31.0|         1.11|
|  Victor Boniface|de Bundesliga|    Leverkusen|     22.0|         0.96|
|      Deniz Undav|de Bundesliga|     Stuttgart|     28.0|         0.93|
|      Loïs Openda|de Bundesliga|    RB Leipzig|     31.0|         0.91|
|Ermedin Demirović|de Bundesliga|      Augsburg|     24.0|         0.73|
|       Leroy Sané|de Bundesliga| Bayern Munich|     19.0|          0.7|
|  Andrej Kramarić|de Bundesliga|    Hoffenheim|     21.0|          0.7|
|    Álex Grimaldo|de Bundesliga|    Leverkusen|     23.0|          0.7|
|  Niclas Füllkrug|de Bundesliga|      Dortmund|     20.0|         0.69|
|    Florian Wirtz|de Bundesliga|    Leverkusen|   

In [51]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, col, round, sum,row_number

# Window for dense ranking within each league
rankedWindow = Window.partitionBy("Comp").orderBy(col("G+A_per_match").desc())

# Dense rank
rankedDF = groupedResultsDF.select(
    col("Player"),
    col("Squad"),
    col("Comp"),
    col("total_G+A"),
    col("G+A_per_match"),
    dense_rank().over(rankedWindow).alias("rank")
)


rankedDF.show(20)


+-----------------+--------------+-------------+---------+-------------+----+
|           Player|         Squad|         Comp|total_G+A|G+A_per_match|rank|
+-----------------+--------------+-------------+---------+-------------+----+
|       Harry Kane| Bayern Munich|de Bundesliga|     44.0|         1.38|   1|
|  Serhou Guirassy|     Stuttgart|de Bundesliga|     31.0|         1.11|   2|
|  Victor Boniface|    Leverkusen|de Bundesliga|     22.0|         0.96|   3|
|      Deniz Undav|     Stuttgart|de Bundesliga|     28.0|         0.93|   4|
|      Loïs Openda|    RB Leipzig|de Bundesliga|     31.0|         0.91|   5|
|Ermedin Demirović|      Augsburg|de Bundesliga|     24.0|         0.73|   6|
|       Leroy Sané| Bayern Munich|de Bundesliga|     19.0|          0.7|   7|
|  Andrej Kramarić|    Hoffenheim|de Bundesliga|     21.0|          0.7|   7|
|    Álex Grimaldo|    Leverkusen|de Bundesliga|     23.0|          0.7|   7|
|  Niclas Füllkrug|      Dortmund|de Bundesliga|     20.0|      

In [52]:
# Top 2 players in each league(filtering)
topPlayersDF = rankedDF.filter(col("rank") <= 3)

# Top 2 players per league
topPlayersDF.show()


+--------------------+---------------+------------------+---------+-------------+----+
|              Player|          Squad|              Comp|total_G+A|G+A_per_match|rank|
+--------------------+---------------+------------------+---------+-------------+----+
|          Harry Kane|  Bayern Munich|     de Bundesliga|     44.0|         1.38|   1|
|     Serhou Guirassy|      Stuttgart|     de Bundesliga|     31.0|         1.11|   2|
|     Victor Boniface|     Leverkusen|     de Bundesliga|     22.0|         0.96|   3|
|      Erling Haaland|Manchester City|eng Premier League|     32.0|         1.03|   1|
|         Cole Palmer|        Chelsea|eng Premier League|     33.0|          1.0|   2|
|       Mohamed Salah|      Liverpool|eng Premier League|     28.0|         0.88|   3|
|     Jude Bellingham|    Real Madrid|        es La Liga|     25.0|         0.89|   1|
|        Artem Dovbyk|         Girona|        es La Liga|     32.0|         0.89|   1|
|   Alexander Sørloth|     Villarreal|     

In [53]:
# Window with a tie-breaking rule
rankedWindow = Window.partitionBy("Comp").orderBy(
    col("G+A_per_match").desc(),  # Primary: G+A per match
    col("total_G+A").desc()       # Secondary: Total G+A
)

# Row_number for strict ranking
strictRankedDF = groupedResultsDF.select(
    col("Player"),
    col("Squad"),
    col("Comp"),
    col("total_G+A"),
    col("G+A_per_match"),
    row_number().over(rankedWindow).alias("rank")
)

# Top 3 players in each league (filtering)
top3PlayersDF = strictRankedDF.filter(col("rank") <= 3)

# Top 3 players per league
top3PlayersDF.show()

+--------------------+---------------+------------------+---------+-------------+----+
|              Player|          Squad|              Comp|total_G+A|G+A_per_match|rank|
+--------------------+---------------+------------------+---------+-------------+----+
|          Harry Kane|  Bayern Munich|     de Bundesliga|     44.0|         1.38|   1|
|     Serhou Guirassy|      Stuttgart|     de Bundesliga|     31.0|         1.11|   2|
|     Victor Boniface|     Leverkusen|     de Bundesliga|     22.0|         0.96|   3|
|      Erling Haaland|Manchester City|eng Premier League|     32.0|         1.03|   1|
|         Cole Palmer|        Chelsea|eng Premier League|     33.0|          1.0|   2|
|       Mohamed Salah|      Liverpool|eng Premier League|     28.0|         0.88|   3|
|        Artem Dovbyk|         Girona|        es La Liga|     32.0|         0.89|   1|
|     Jude Bellingham|    Real Madrid|        es La Liga|     25.0|         0.89|   2|
|   Alexander Sørloth|     Villarreal|     

In [54]:
# Sorting players by g+a_per_match in descending order
bestPlayersDF = top3PlayersDF.orderBy(
    col("G+A_per_match").desc()  # Sort by g+a_per_match in descending order
)

# Show the results
bestPlayersDF.show(20)


+--------------------+---------------+------------------+---------+-------------+----+
|              Player|          Squad|              Comp|total_G+A|G+A_per_match|rank|
+--------------------+---------------+------------------+---------+-------------+----+
|          Harry Kane|  Bayern Munich|     de Bundesliga|     44.0|         1.38|   1|
|       Kylian Mbappé|      Paris S-G|        fr Ligue 1|     34.0|         1.17|   1|
|     Serhou Guirassy|      Stuttgart|     de Bundesliga|     31.0|         1.11|   2|
|      Erling Haaland|Manchester City|eng Premier League|     32.0|         1.03|   1|
|         Cole Palmer|        Chelsea|eng Premier League|     33.0|          1.0|   2|
|     Victor Boniface|     Leverkusen|     de Bundesliga|     22.0|         0.96|   3|
|        Artem Dovbyk|         Girona|        es La Liga|     32.0|         0.89|   1|
|     Jude Bellingham|    Real Madrid|        es La Liga|     25.0|         0.89|   2|
|       Mohamed Salah|      Liverpool|eng P

In [56]:
#
bucket = "temp_assignment2" 
spark.conf.set('temporaryGcsBucket', bucket)

bestPlayersDF.write.csv("gs://temp_assignment2/best_players.csv", header=True)

# Saving DF to BigQuery
bestPlayersDF.write.format('bigquery') \
  .option('table', 'de-assignment2-group10.Pipeline_player_stats.best_players') \
  .mode("overwrite") \
  .save()


In [57]:
# Stop the spark context
spark.stop()